In [13]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.utils as vutils
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable


In [229]:
class Network(nn.Module):
    
    def __init__(self, input_size=(3,32,32), n_out=10):
        super(Network, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,32,3),
            nn.ReLU(),
            nn.Conv2d(32,32,3),
            nn.ReLU(),
            nn.MaxPool2d((3,3))
        )
        self.flat_fts = self.get_flat_fts(input_size, self.features)
        self.classifier1 = nn.Sequential(
            nn.Linear(self.flat_fts, 100),
            nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(100,1),
            nn.Sigmoid()
        )
        self.classifier2 = nn.Sequential(
            nn.Linear(self.flat_fts, 100),
            nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(100,n_out),
            nn.Softmax()
        )
    
    def get_flat_fts(self, in_size, fts):
        f = fts(Variable(torch.ones(1,*in_size)))
        return int(np.prod(f.size()[1:]))
    
    def forward(self, x):
        fts = self.features(x)
        flat_fts = fts.view(-1, self.flat_fts)
        out1 = self.classifier1(flat_fts)
        out2 = self.classifier2(flat_fts)
        return out1, out2

In [230]:
grads = {}
def save_grad(name):
    def hook(grad):
        grads[name] = grad
    return hook



In [231]:
x  = Variable(torch.randn(2,3,32,32), requires_grad=True)
y1 = Variable(torch.rand(2,1), requires_grad=False)
y2 = Variable(torch.randperm(10)[:2], requires_grad=False)


In [240]:
net = Network()
net.zero_grad()

out1, out2 = net(x)

out1.register_hook(save_grad('out1'))
out2.register_hook(save_grad('out2'))

out1.size(), out2.size()

(torch.Size([2, 1]), torch.Size([2, 10]))

In [241]:
criterion1 = nn.BCELoss()
criterion2 = nn.CrossEntropyLoss()

loss1 = criterion1(out1, y1)
loss2 = criterion2(out2, y2)

loss1.register_hook(save_grad('loss1'))
loss2.register_hook(save_grad('loss2'))

lam = 0.5
loss = loss1 + lam * loss2
loss.register_hook(save_grad('loss'))


In [242]:
loss.backward()

In [235]:
loss1.data, loss2.data

(
  0.6935
 [torch.FloatTensor of size 1], 
  2.2977
 [torch.FloatTensor of size 1])

In [236]:
loss.grad

In [245]:
grads['loss2']

Variable containing:
 0.5000
[torch.FloatTensor of size 1]

In [238]:
grads['out2']

Variable containing:
 0.0251  0.0248  0.0246  0.0249  0.0257 -0.2247  0.0250  0.0250  0.0248  0.0250
 0.0252  0.0249  0.0245  0.0248  0.0255  0.0251 -0.2250  0.0249  0.0250  0.0250
[torch.FloatTensor of size 2x10]

In [193]:
torch.autograd.backward(x, [out1.grad, out2.grad])

RuntimeError: element 595291 of gradients tuple is None, but the corresponding Variable requires grad

In [226]:
grads = {}
def save_grad(name):
    def hook(grad):
        grads[name] = grad
    return hook

x = Variable(torch.randn(1,1), requires_grad=True)
y = 3*x
z = y**2

# In here, save_grad('y') returns a hook (a function) that keeps 'y' as name
y.register_hook(save_grad('y'))
z.register_hook(save_grad('z'))
z.backward()

print(grads['y'])
print(grads['z'])

Variable containing:
 10.0909
[torch.FloatTensor of size 1x1]

Variable containing:
 1
[torch.FloatTensor of size 1x1]

